In [72]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ast import literal_eval
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import sympy as sym
from sympy.parsing import mathematica

In [73]:

RESULTS_DIR = "final_results"
benchmarks = [
    "rational/plain",
    "rational_2/plain",
    "sqrt/plain",
    "rational_sqrt/plain",

    "rational/with_constant",
    "rational_2/with_constant",
    "sqrt/with_constant",
    "rational_sqrt/with_constant",
]

In [74]:
def process_output(row):
    if pd.isnull(row):
        return (None, None)

    try:
        row = float(row)
        return (row, row)
    except:
        pass

    try:
        return literal_eval(str(row).strip())
    except Exception as e:
        pass
    
    try:
        for w in ['integrate', 'boole']:
            if w in row.lower():
                return (None, None)        
        
        row = row.replace("[",'(').replace("]", ")").replace("L", "l").replace("E(p_) =", "").replace("^", "**").strip()
        row = sym.parse_expr(row).evalf()
        
        return (row,row)
        
    except Exception as e:
        print(e, row)
    
        
    return (None, None)

In [75]:
def process_time(row):
    if pd.isnull(row):
        return None

    try:
        row = float(row)
        return row
    except:
        pass

    try:
        return literal_eval(str(row).strip())[0]
    except Exception as e:
        pass
        
    except Exception as e:
        print(e, row)
    
        
    return None

In [76]:
def process_benchmark(benchmark):
    df_list = []
    for f_name in os.listdir(os.path.join(RESULTS_DIR, benchmark)):
        tool_name =  f_name.split("_")[-2]
        df_ = pd.read_csv(os.path.join(RESULTS_DIR, benchmark, f_name))
        df_['tool'] = tool_name
        df_list.append(df_[['index', 'tool','output', 'time']])
    df = pd.concat(df_list, ignore_index=True)

    df['output'] = df['output'].apply(process_output)
    df['time'] = df['time'].apply(process_time)
    
    df['lower'], df['upper'] = df['output'].apply(lambda o: o[0]),df['output'].apply(lambda o: o[1])
    df['error'] = df['upper']-df['lower']
    df['error_norm'] = MinMaxScaler().fit_transform(df['error'].values.reshape(-1,1))
    df['benchmark'] = benchmark
    df['is_solved'] = (~pd.isnull(df['error']))&(df['time']<=3600)
    return df

In [77]:
df_b_list = []
for b in benchmarks:
    df_b_list.append(process_benchmark(b))

df_b = pd.concat(df_b_list)

In [78]:
# df_b[(df_b.benchmark=='rational/with_constant')].sort_values('error_norm')

In [79]:
prop = 'time'
method = 'mean'

df_ = df_b[[prop, 'tool', 'benchmark']].dropna().groupby(['benchmark','tool']).agg({prop:[method]}).reset_index()
df_.columns = df_.columns.droplevel(1)
df_ = df_.pivot_table(columns='tool', index='benchmark', values=prop)
df_ = df_.loc[benchmarks]
df_

tool,faza,gubpi,latte,psi,volesti
benchmark,,,,,
rational/plain,41.312201,0.492676,0.061535,17.151220,0.109235
rational_2/plain,894.456383,0.658119,0.043645,NaN,0.059738
sqrt/plain,17.520717,0.439463,0.000049,0.420083,0.000055
rational_sqrt/plain,67.533403,0.519594,0.000047,6.869766,0.000048
rational/with_constant,41.865961,0.456974,0.066416,34.701984,0.107140
rational_2/with_constant,NaN,0.637626,0.053168,NaN,0.111224
sqrt/with_constant,51.052686,0.385814,0.000055,0.603179,0.000053
rational_sqrt/with_constant,123.853678,0.419659,0.000044,167.749268,0.000050


In [ ]:
df_.to_csv(os.path.join(RESULTS_DIR, 'agg', f'{prop}_{method}.csv'))